In [ ]:
# imports
import bz2
import pickle5 as pickle
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# functions
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = pickle.load(data)
    return data

In [ ]:


df = decompress_pickle('6nutrients.pbz2')



In [19]:
def score_diff(id, shiftid, diff, shiftdiff):
    if id!=shiftid or shiftid==pd.isnull:
        return -1
    elif diff>=0 and shiftdiff>=0:
        return 1
    else:
        return 0

df1 = df.copy()

days = 10

for day in range(1,days):

    if day > 2:
        # we can drop old columns to save some space
        df1 = df1.drop(columns=['difference'+str(day-2),'shiftid'+str(day-1),'shiftdiff'+str(day-1)])


    if day > 1:
        # shift the difference
        df1.insert(1, 'difference'+str(day-1), df1.shift(periods=-day, axis=0)['difference0'])
    else:
        df1['difference'+str(day-1)] = df.apply(lambda x: x['total.calories'] - x['goal.calories'], axis=1)

    # remove rows with only one day worth of data
    df1 = df1.groupby('id').filter(lambda x : len(x)>days)



    # add shiftid column
    df1.insert(1, 'shiftid'+str(day), df1.shift(periods=-day, axis=0)['id'])

    df1['shiftdiff'+str(day)] =  df1.shift(periods=-day, axis=0)['difference'+str(day-1)]
    df1['new_success'+str(day)] = df1.apply(lambda x: score_diff(x['id'],x['shiftid'+str(day)],x['difference'+str(day-1)],x['shiftdiff'+str(day)]), axis=1)

    print("Day: "+str(day))
    df1['new_success'+str(day)].value_counts()

Day: 1
Day: 2
Day: 3
Day: 4
Day: 5
Day: 6
Day: 7
Day: 8
Day: 9


 0    454197
 1     49506
-1      6846
Name: new_success1, dtype: int64

 0    450477
 1     46380
-1     13692
Name: new_success2, dtype: int64

 0    445476
 1     44535
-1     20538
Name: new_success3, dtype: int64

 0    439752
 1     43413
-1     27384
Name: new_success4, dtype: int64

 0    433722
 1     42597
-1     34230
Name: new_success5, dtype: int64

 0    427163
 1     42310
-1     41076
Name: new_success6, dtype: int64

 0    420014
-1     47922
 1     42613
Name: new_success7, dtype: int64

 0    415372
-1     54768
 1     40409
Name: new_success8, dtype: int64

 0    410132
-1     61614
 1     38803
Name: new_success9, dtype: int64

In [ ]:
#